In [28]:
import pandas as pd
import numpy as nm

In [53]:
# read the data
d1=pd.read_excel('D:\GoogleDrive\custom Codes\Mar18\Rating\Loan_Data2.xlsx')
# see the dimension
d1.shape

(98179, 40)

In [54]:
# bring customer group id where customer id is not present
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].fillna(d1['CUSTOMER_GROUP_ID'])

# turn the customer id into string
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].astype(str)

# make the leading zero and turn customer id into 10 digits
d1['CUSTOMER_ID']=d1['CUSTOMER_ID'].apply(lambda x: x.zfill(12)).str[:10]
d1.head()

,BRANCH_ID,BRANCH_NM,CUSTOMER_ID,CUSTOMER_GROUP_ID,ACCOUNT_NO,ACCOUNT_TITLE,PROPOSAL_ID,COMMITMENT_ID,CUSTOMER_CATEGORY_ID,CUSTOMER_CONCENTRATION_ID,...,COMMIT_COL_VALUE,COLLATERAL_TYPE_ID,COLLATERAL_TYPE_NM,COLLATERAL_ID,COLLATERAL_CATEGORY,COLLATERAL_DESCRIP,COLLATERAL_FV_MV_VALUE,ELIGIBLE_SECURITY_VALUE,INTEREST_SUSPENSE,ACCOUNT_STATUS_ID
1,1271,Local Office,0000004607,NaN,63900000002,MD. YEAKUB MOJUMDER,280491,172197,1.0,8.0,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1545,Corporate Branch,0000004638,NaN,63900005929,MD SIDDIQUR RAHMAN,265511,165547,1.0,1.0,...,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1271,Local Office,0000005131,NaN,61306116836,AMBIA AKTER,350725,204689,1.0,1.0,...,169000.00,3.0,Lien/Pledge of Deposits (Earmark Funds),793.0,"BLOCK OF DEPOSIT A/C(FDR,MMP,DBDS,SCHEMES)",MDS No.303-12-332 dt.07.01.2009 having Tk.1.30...,169000.00,169000.00,NaN,1
4,1271,Local Office,0000005182,NaN,61306116801,NAZMES SABAH SHARIFA BEGUM LIZ A SABAH SHARIFA...,128308,98518,1.0,1.0,...,189000.00,3.0,Lien/Pledge of Deposits (Earmark Funds),95.0,"BLOCK OF DEPOSIT A/C(FDR,MMP,DBDS,SCHEMES)",Lien amount of MDS-390 for total Tk.1.89 lac,189000.00,189000.00,NaN,1
5,1271,Local Office,0000005206,NaN,61306116829,AKBAR HOSSAIN,307307,184292,1.0,1.0,...,149320.08,3.0,Lien/Pledge of Deposits (Earmark Funds),550.0,"BLOCK OF DEPOSIT A/C(FDR,MMP,DBDS,SCHEMES)",Lien of MDS for Tk.0.98 lac,149320.08,149320.08,NaN,1


In [55]:
# make unique id by concating branch id and collateral id
d1['UI']=d1['BRANCH_ID'].astype(str)+d1['COLLATERAL_ID'].astype(str)

In [56]:
# drop duplicate rows by the new column UI
d1=d1.drop_duplicates(subset='UI', keep="first")
d1.shape

(55745, 41)

In [57]:
# fill the na is eligible collateral value with 0
d1['ELIGIBLE_SECURITY_VALUE']=d1['ELIGIBLE_SECURITY_VALUE'].fillna(0)

In [58]:
# now group by sum for each customer id
do=d1.groupby(['CUSTOMER_ID'])['ELIGIBLE_SECURITY_VALUE'].agg(sum)
do=do.reset_index()
do.head()
do['ELIGIBLE_SECURITY_VALUE'].sum()

183500370904.08997

In [59]:
# now the data with actual provision
d2=pd.read_excel('D:\GoogleDrive\custom Codes\Mar18\Rating\Loan_D1.xlsx')
d2.columns

Index(['BRANCH_ID', 'BRANCH_NM', 'CUSTOMER_ID', 'CUSTOMER_GROUP_ID',
       'ACCOUNT_NO', 'ACCOUNT_TITLE', 'PROPOSAL_ID', 'COMMITMENT_ID',
       'PRODUCT_ID', 'SANCTION_DATE', 'DISBURSEMENT_DT', 'EXPIRY_DT',
       'EFFECTIVE_DT', 'LIMIT_AMT', 'TOTAL_DISBURSEMENT', 'ECON_PURP_SBS_ID',
       'CL_STATUS_ID', 'BALANCE_LCY', 'CLOSE_BALA_LCY', 'CLOSING_OS_BAL_LCY',
       'SBS_SECTOR_CODE', 'SBS_EP_CODE', 'SBS_SECURITY_CODE', 'SBS_PRD_CODE',
       'SBS_BILL_CODE', 'COMMIT_COL_VALUE'],
      dtype='object')

In [60]:
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].fillna(d2['CUSTOMER_GROUP_ID'])
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].astype(str).apply(lambda x: x.zfill(12))
d2['CUSTOMER_ID']=d2['CUSTOMER_ID'].str[:10]

In [61]:
do.head()

,CUSTOMER_ID,ELIGIBLE_SECURITY_VALUE
0,0000000342,762500.0
1,0000000531,7785000.0
2,0000000642,2000000.0
3,0000000751,5309000.0
4,0000000752,4023000.0


In [68]:
# customer id wise total outstanding
du=d2.groupby(['CUSTOMER_ID'])['CLOSE_BALA_LCY'].agg(sum).reset_index()

# rename columns
du=du.rename(columns={'CLOSE_BALA_LCY':'TOTAL_OUTSTANDING'})
du.head()

,CUSTOMER_ID,TOTAL_OUTSTANDING
0,0000000342,646505.57
1,0000000531,1222290.46
2,0000000642,91771.88
3,0000000751,104876.85
4,0000000752,470921.03


In [69]:
print(du['TOTAL_OUTSTANDING'].sum())
print(du.shape)

102567357006.81999
(68085, 2)


In [80]:
# select the two columns
dn=d2[['CUSTOMER_ID', 'CLOSE_BALA_LCY']]

#mege the actual outstanding with the total outstanding per customer id
dt=pd.merge(dn, du,  on='CUSTOMER_ID', how='left')
dt.shape
dt['TOTAL_OUTSTANDING'].sum()
dt.head()
print(dt['CLOSE_BALA_LCY'].sum())

102567357006.82


In [72]:
# merge total collateral data with total outstanding and actual outstanding
dm=pd.merge(dt, do, on='CUSTOMER_ID', how='left')
dm.shape

(76098, 4)

In [83]:
# now calculate collateral value for each account (based on its outstanding)
dm['COL_PER_AC']=dm['ELIGIBLE_SECURITY_VALUE']*dm['CLOSE_BALA_LCY']/dm['TOTAL_OUTSTANDING']
dm.head()
dm['COL_PER_AC'].sum()

175834104542.97